<a href="https://colab.research.google.com/github/aklapatch/435-finalproj/blob/master/UpscaleFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


NameError: ignored

In [2]:
import os, re, math, json, shutil, pprint
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import IPython.display as display
import numpy as np
import tensorflow as tf
from skimage import data, io, filters
from matplotlib import pyplot as plt
from numpy import array
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model, load_model

tf.enable_eager_execution()


Using TensorFlow backend.


In [0]:
#@title visualization utilities [RUN ME]
"""
This cell contains helper functions used for visualization
and downloads only. You can skip reading it. There is very
little useful Keras/Tensorflow code here.
"""

# Matplotlib config
plt.ioff()
plt.rc('image', cmap='gray_r')
plt.rc('grid', linewidth=1)
plt.rc('xtick', top=False, bottom=False, labelsize='large')
plt.rc('ytick', left=False, right=False, labelsize='large')
plt.rc('axes', facecolor='F8F8F8', titlesize="large", edgecolor='white')
plt.rc('text', color='a8151a')
plt.rc('figure', facecolor='F0F0F0', figsize=(16,9))
# Matplotlib fonts
MATPLOTLIB_FONT_DIR = os.path.join(os.path.dirname(plt.__file__), "mpl-data/fonts/ttf")

# pull a batch from the datasets. This code is not very nice, it gets much better in eager mode (TODO)
def dataset_to_numpy_util(training_dataset, validation_dataset, N):
  
  # get one batch from each: 10000 validation digits, N training digits
  batch_train_ds = training_dataset.apply(tf.data.experimental.unbatch()).batch(N)
  
  # eager execution: loop through datasets normally
  if tf.executing_eagerly():
    for validation_digits, validation_labels in validation_dataset:
      validation_digits = validation_digits.numpy()
      validation_labels = validation_labels.numpy()
      break
    for training_digits, training_labels in batch_train_ds:
      training_digits = training_digits.numpy()
      training_labels = training_labels.numpy()
      break
    
  else:
    v_images, v_labels = validation_dataset.make_one_shot_iterator().get_next()
    t_images, t_labels = batch_train_ds.make_one_shot_iterator().get_next()
    # Run once, get one batch. Session.run returns numpy results
    with tf.Session() as ses:
      (validation_digits, validation_labels,
       training_digits, training_labels) = ses.run([v_images, v_labels, t_images, t_labels])
  
  # these were one-hot encoded in the dataset
  validation_labels = np.argmax(validation_labels, axis=1)
  training_labels = np.argmax(training_labels, axis=1)
  
  return (training_digits, training_labels,
          validation_digits, validation_labels)

# create digits from local fonts for testing
def create_digits_from_local_fonts(n):
  font_labels = []
  img = PIL.Image.new('LA', (28*n, 28), color = (0,255)) # format 'LA': black in channel 0, alpha in channel 1
  font1 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'DejaVuSansMono-Oblique.ttf'), 25)
  font2 = PIL.ImageFont.truetype(os.path.join(MATPLOTLIB_FONT_DIR, 'STIXGeneral.ttf'), 25)
  d = PIL.ImageDraw.Draw(img)
  for i in range(n):
    font_labels.append(i%10)
    d.text((7+i*28,0 if i<10 else -4), str(i%10), fill=(255,255), font=font1 if i<10 else font2)
  font_digits = np.array(img.getdata(), np.float32)[:,0] / 255.0 # black in channel 0, alpha in channel 1 (discarded)
  font_digits = np.reshape(np.stack(np.split(np.reshape(font_digits, [28, 28*n]), n, axis=1), axis=0), [n, 28*28])
  return font_digits, font_labels

# utility to display a row of digits with their predictions
def display_digits(digits, predictions, labels, title, n):
  fig = plt.figure(figsize=(13,3))
  digits = np.reshape(digits, [n, 28, 28])
  digits = np.swapaxes(digits, 0, 1)
  digits = np.reshape(digits, [28, 28*n])
  plt.yticks([])
  plt.xticks([28*x+14 for x in range(n)], predictions)
  plt.grid(b=None)
  for i,t in enumerate(plt.gca().xaxis.get_ticklabels()):
    if predictions[i] != labels[i]: t.set_color('red') # bad predictions in red
  plt.imshow(digits)
  plt.grid(None)
  plt.title(title)
  display.display(fig)
  
# utility to display multiple rows of digits, sorted by unrecognized/recognized status
def display_top_unrecognized(digits, predictions, labels, n, lines):
  idx = np.argsort(predictions==labels) # sort order: unrecognized first
  for i in range(lines):
    display_digits(digits[idx][i*n:(i+1)*n], predictions[idx][i*n:(i+1)*n], labels[idx][i*n:(i+1)*n],
                   "{} sample validation digits out of {} with bad predictions in red and sorted first".format(n*lines, len(digits)) if i==0 else "", n)

def plot_learning_rate(lr_func, epochs):
  xx = np.arange(epochs+1, dtype=np.float)
  y = [lr_decay(x) for x in xx]
  fig, ax = plt.subplots(figsize=(9, 6))
  ax.set_xlabel('epochs')
  ax.set_title('Learning rate\ndecays from {:0.3g} to {:0.3g}'.format(y[0], y[-2]))
  ax.minorticks_on()
  ax.grid(True, which='major', axis='both', linestyle='-', linewidth=1)
  ax.grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
  ax.step(xx,y, linewidth=3, where='post')
  display.display(fig)

class PlotTraining(tf.keras.callbacks.Callback):
  def __init__(self, sample_rate=1, zoom=1):
    self.sample_rate = sample_rate
    self.step = 0
    self.zoom = zoom
    self.steps_per_epoch = 60000//BATCH_SIZE

  def on_train_begin(self, logs={}):
    self.batch_history = {}
    self.batch_step = []
    self.epoch_history = {}
    self.epoch_step = []
    self.fig, self.axes = plt.subplots(1, 2, figsize=(16, 7))
    plt.ioff()

  def on_batch_end(self, batch, logs={}):
    if (batch % self.sample_rate) == 0:
      self.batch_step.append(self.step)
      for k,v in logs.items():
        # do not log "batch" and "size" metrics that do not change
        # do not log training accuracy "acc"
        if k=='batch' or k=='size':# or k=='acc':
          continue
        self.batch_history.setdefault(k, []).append(v)
    self.step += 1

  def on_epoch_end(self, epoch, logs={}):
    plt.close(self.fig)
    self.axes[0].cla()
    self.axes[1].cla()
      
    self.axes[0].set_ylim(0, 1.2/self.zoom)
    self.axes[1].set_ylim(1-1/self.zoom/2, 1+0.1/self.zoom/2)
    
    self.epoch_step.append(self.step)
    for k,v in logs.items():
      # only log validation metrics
      if not k.startswith('val_'):
        continue
      self.epoch_history.setdefault(k, []).append(v)

    display.clear_output(wait=True)
    
    for k,v in self.batch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.batch_step) / self.steps_per_epoch, v, label=k)
      
    for k,v in self.epoch_history.items():
      self.axes[0 if k.endswith('loss') else 1].plot(np.array(self.epoch_step) / self.steps_per_epoch, v, label=k, linewidth=3)
      
    self.axes[0].legend()
    self.axes[1].legend()
    self.axes[0].set_xlabel('epochs')
    self.axes[1].set_xlabel('epochs')
    self.axes[0].minorticks_on()
    self.axes[0].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[0].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    self.axes[1].minorticks_on()
    self.axes[1].grid(True, which='major', axis='both', linestyle='-', linewidth=1)
    self.axes[1].grid(True, which='minor', axis='both', linestyle=':', linewidth=0.5)
    display.display(self.fig)

Utility functions

In [0]:
def load_path(path):
    directories = []
    if os.path.isdir(path):
        directories.append(path)
    for elem in os.listdir(path):
        if os.path.isdir(os.path.join(path,elem)):
            directories = directories + load_path(os.path.join(path,elem))
            directories.append(os.path.join(path,elem))
    return directories

def normalize(input_data):

    return (input_data.astype(np.float32) - 127.5)/127.5 
    
def denormalize(input_data):
    input_data = (input_data + 1) * 127.5
    return input_data.astype(np.uint8)

def load_data_from_dirs(dirs, ext,num_files):
    files = []
    file_names = []
    count = 0
    for d in dirs:
        for f in os.listdir(d): 
            if f.endswith(ext):
                image = io.imread(os.path.join(d,f))
                if len(image.shape) > 2:
                    files.append(image)
                    if len(files) >= num_files:
                        return files
                    file_names.append(os.path.join(d,f))
                count = count + 1
    return files 

def load_training_data(directory, ext, number_of_images = 1000):    
    files = load_data_from_dirs(load_path(directory), ext, number_of_images)
    
    if len(files) < number_of_images:
        print("Number of image files are less then you specified")
        print("Please reduce number of images to %d" % len(files))
        sys.exit()
        
    test_array = array(files)
    if len(test_array.shape) < 3:
        print("Images are of not same shape")
        print("Please provide same shape images")
        sys.exit()
    
    return test_array




Loading data from Drive


In [0]:

imgnum = 800
target_imgs = load_training_data("/content/drive/My Drive/ALOT384/","jpg", imgnum)
input_imgs = load_training_data("/content/drive/My Drive/ALOTdown&UpScaled/", "jpg", imgnum)

Init the Model


In [0]:
inputTensor = Input(shape=(384,384,3))
x = UpSampling2D(size=(2,2),data_format="channels_last",interpolation="bilinear")(inputTensor)
x = Conv2D(3,(5,5),data_format="channels_last",activation='relu',padding="same")(x)
x = MaxPooling2D((2,2), padding='same')(x)
x = Conv2D(3, (5,5),data_format="channels_last", activation='relu', padding='same')(x)

model = Model(inputTensor, x)
model.compile(optimizer='sgd', loss='mean_absolute_error', metrics=['accuracy'])


data set of 500 images
### Optimizers
*   SGD: about .58 acc and does not get better
*   Adam: worse than sgd

### Misc
- a 5,5 convolution is better than 3,3 it seems
- but a 7,7 is not really better than a 5,5
- a 9,9 takes longer to get to .578 acc, but it gets there, It trains much slower
- batch size does not affect accuracy it seems
- changing image sample size to 800 kicks the acc down to .16 at the beginning
- using 2 Conv2D filters seems to only be able to get .578 acc at max

Fit the Model

In [20]:
plot = PlotTraining(sample_rate=10, zoom=1)

# the number of epochs
epochs = 1000
pocsperloop=100
eps = int(epochs/pocsperloop)
for e in range(1, eps+1):
  print("iteration %2d/%2d" % (e, eps))
  model.fit(input_imgs, 
            target_imgs, 
            epochs=pocsperloop,
            batch_size=128,
            shuffle=True,
            validation_split=0.05
            callbacks=[plot])
  print("saving model")
  model.save("/content/drive/My Drive/models/last_model.h5")

model.summary()

iteration  1/1000
Train on 760 samples, validate on 40 samples
Epoch 1/100
760/760 [==============================] - 3s 4ms/step - loss: 81.2116 - acc: 0.1681 - val_loss: 40.2586 - val_acc: 0.1556
Epoch 2/100
760/760 [==============================] - 2s 3ms/step - loss: 81.2014 - acc: 0.1681 - val_loss: 40.2492 - val_acc: 0.1556
Epoch 3/100
760/760 [==============================] - 2s 3ms/step - loss: 81.1913 - acc: 0.1681 - val_loss: 40.2397 - val_acc: 0.1556
Epoch 4/100
760/760 [==============================] - 2s 3ms/step - loss: 81.1811 - acc: 0.1681 - val_loss: 40.2303 - val_acc: 0.1556
Epoch 5/100
760/760 [==============================] - 2s 3ms/step - loss: 81.1710 - acc: 0.1681 - val_loss: 40.2208 - val_acc: 0.1556
Epoch 6/100
760/760 [==============================] - 2s 3ms/step - loss: 81.1607 - acc: 0.1681 - val_loss: 40.2114 - val_acc: 0.1556
Epoch 7/100
760/760 [==============================] - 2s 3ms/step - loss: 81.1507 - acc: 0.1681 - val_loss: 40.2020 - val_acc:

NameError: ignored

load if a model if you want


In [0]:
# load if you can
model = load_model("/content/drive/My Drive/models/last_model.h5")

save model

In [0]:
model.save("/content/drive/My Drive/models/last_model.h5")